# 准备数据集

nltk分词, jieba分词，构建词典，数字编码，形成batch

## 导入包

In [1]:
import os
import sys
import math

from collections import Counter
import numpy as np
import random 

import torch
import torch.nn as nn
import torch.nn.functional as F

import nltk
# 注意这里需要下载一个punkt包
# nltk.download()

import jieba

## 导入数据并分词

先导入数据：
- train.txt和dev.txt
- 每个文件多行，每一行都是英文句子，后面跟着中文翻译，两者用'\t'分开

下面的代码导入数据，已经进行了分词处理，每个句子前后都加了两个特殊符号，和原来给的不一样的地方是尝试了jieba分词，而不是单个汉字

In [2]:
def load_data(in_file):
    cn = []
    en = []
    num_examples = 0
    with open(in_file, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.strip().split('\t')      # 每一行是英文+翻译的形式
            #print(line)   # ['Anyone can do that.', '任何人都可以做到。']
            #print(nltk.word_tokenize(line[0].lower()))    # ['anyone', 'can', 'do', 'that', '.']
            en.append(['BOS'] + nltk.word_tokenize(line[0].lower()) + ['EOS'])
            #print([c for c in line[1]])   ['任', '何', '人', '都', '可', '以', '做', '到', '。']
            #print(list(jieba.cut(line[1])))        ['任何人', '都', '可以', '做到', '。']
            #cn.append(['BOS'] + [c for c in line[1]] + ['EOS'])
            cn.append(['BOS'] + list(jieba.cut(line[1])) + ['EOS'])
    return en, cn

train_file = 'data/nmt/en-cn/train.txt'
dev_file = 'data/nmt/en-cn/dev.txt'
train_en, train_cn = load_data(train_file)
dev_en, dev_cn = load_data(dev_file)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Huris\AppData\Local\Temp\jieba.cache
Loading model cost 0.465 seconds.
Prefix dict has been built successfully.


看一下长什么样，每句话分成了一个个单词，接下来会把这些单词对应到词典中的位置上去，用词典中的位置表示这些句子，因为计算机只认识数字，不认识这些汉字

In [3]:
print(train_en[:3], train_cn[:3])
print(dev_en[:3], dev_cn[:3])

[['BOS', 'anyone', 'can', 'do', 'that', '.', 'EOS'], ['BOS', 'how', 'about', 'another', 'piece', 'of', 'cake', '?', 'EOS'], ['BOS', 'she', 'married', 'him', '.', 'EOS']] [['BOS', '任何人', '都', '可以', '做到', '。', 'EOS'], ['BOS', '要', '不要', '再來', '一塊', '蛋糕', '？', 'EOS'], ['BOS', '她', '嫁给', '了', '他', '。', 'EOS']]
[['BOS', 'she', 'put', 'the', 'magazine', 'on', 'the', 'table', '.', 'EOS'], ['BOS', 'hey', ',', 'what', 'are', 'you', 'doing', 'here', '?', 'EOS'], ['BOS', 'please', 'keep', 'this', 'secret', '.', 'EOS']] [['BOS', '她', '把', '雜誌', '放在', '桌上', '。', 'EOS'], ['BOS', '嘿', '，', '你', '在', '這做', '什麼', '？', 'EOS'], ['BOS', '請', '保守', '這個', '秘密', '。', 'EOS']]


## 构建单词表

遍历每个句子，统计每个单词出现的个数，按照频率由高到低的顺序把单词放到字典中，并且建立一种字典索引到具体词的一种映射关系。

有了字典之后，就可以把上面的数据用数字进行编码，即它们在字典中的位置表示。

In [4]:
UNK_IDX = 0
PAD_IDX = 1
def build_dict(sentences, max_words=50000):
    word_count = Counter()
    for sentence in sentences:
        for s in sentence:
            word_count[s] += 1
    ls = word_count.most_common(max_words)
    total_words = len(ls) + 2    # 两个特殊的字符UNK和PAD
    word_dict = {w[0]: index + 2 for index, w in enumerate(ls)}   # 字典的前两个位置放特殊字符
    word_dict['UNK'] = UNK_IDX 
    word_dict['PAD'] = PAD_IDX
    return word_dict, total_words

en_dict, en_total_words = build_dict(train_en)
cn_dict, cn_total_words = build_dict(train_cn)

inv_en_dict = {v:k for k, v in en_dict.items()}
inv_cn_dict = {v:k for k, v in cn_dict.items()}

## 数字编码

遍历每个句子，得到它们在字典中的位置，为了后续输入方便，这里还根据句子的长度从短到长进行排序。

In [5]:
def encode(en_sentences, cn_sentences, en_dict, cn_dict, sort_by_len=True):
    length = len(en_sentences)
    out_en_sentences = [[en_dict.get(w, 0) for w in sent] for sent in en_sentences]
    out_cn_sentences = [[cn_dict.get(w, 0) for w in sent] for sent in cn_sentences]
    # 根据英语句子的长度排序
    def len_argsort(seq):   # 这个seq是一个二维矩阵，每一行是一个句子，且都已经用单词在字典中的位置进行了编码
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))
    
    if sort_by_len:
        sorted_index = len_argsort(out_en_sentences)
        out_en_sentences = [out_en_sentences[i] for i in sorted_index]
        out_cn_sentences = [out_cn_sentences[i] for i in sorted_index]
    
    return out_en_sentences, out_cn_sentences

train_en, train_cn = encode(train_en, train_cn, en_dict, cn_dict)
dev_en, dev_cn = encode(dev_en, dev_cn, en_dict, cn_dict)

这时候，train_en，train_cn这些里面的每个句子的单词都用数字表示

In [6]:
print(train_en[1])
print(" ".join([inv_cn_dict[i] for i in train_cn[1]]))
print(" ".join([inv_en_dict[i] for i in train_en[1]]))

[2, 1318, 126, 3]
BOS 告辞 ！ EOS
BOS goodbye ! EOS


## 构建batch

数字编码之后，接下来就可以划分好一个个batch了

步骤，一共有三个函数：
- 一个函数是根据batch大小返回batch大小的索引，用于数据集中取具体的句子
- 由于每个句子的长度不一致，因此还需要一个函数对句子进行填充，根据最长长度进行0填充
- 最后一个函数用于生成batch

In [7]:
# 输入训练集大小，batch_size，返回多批连续的batch_size个索引，每个索引代表个样本
# 根据这个索引去拿一个个的batch
def get_minibatches(n, minibatch_size, shuffle=True):
    idx_list = np.arange(0, n, minibatch_size)
    if shuffle:
        np.random.shuffle(idx_list)
    minibatches = []
    for idx in idx_list:
        minibatches.append(np.arange(idx, min(idx + minibatch_size, n)))
    return minibatches      # 返回多批连着的bath_size个索引  
# get_minibatches(len(train_en), 32)

# 数据预处理，每个句子都不等长，通过该函数就可以把句子补齐，不够长的在句子后面添0
def prepare_data(seqs):
    lengths = [len(seq) for seq in seqs]    # 每个句子的长度
    n_samples = len(seqs)       # 一共有多少个句子
    max_len = np.max(lengths)      # 找出最大的句子长度
    
    x = np.zeros((n_samples, max_len)).astype('int32') # 按照最大句子长度生成全0矩阵
    x_lengths = np.array(lengths).astype('int32')
    for idx, seq in enumerate(seqs):   # 把有句子的位置填充进去
        x[idx, :lengths[idx]] = seq
    return x, x_lengths    # x_mask

def gen_examples(en_sentences, cn_sentences, batch_size):
    minibatches = get_minibatches(len(en_sentences), batch_size)  # 得到batch个索引
    all_ex = []
    for minibatch in minibatches:   # 每批数据的索引
        mb_en_sentences = [en_sentences[t] for t in minibatch]   # 取数据
        mb_cn_sentences = [cn_sentences[t] for t in minibatch]  # 取数据
        # 填充成一样的长度，记录句子的真实长度，这个在后面输入网络的时候得用
        mb_x, mb_x_len = prepare_data(mb_en_sentences)
        mb_y, mb_y_len = prepare_data(mb_cn_sentences)
        all_ex.append((mb_x, mb_x_len, mb_y, mb_y_len))
    return all_ex

batch_size = 64
train_data = gen_examples(train_en, train_cn, batch_size)   # 产生训练集
random.shuffle(train_data)
dev_data = gen_examples(dev_en, dev_cn, batch_size)   # 产生验证集

In [8]:
# 看一下batch
print(train_data[1][0].shape, train_data[1][1].shape, train_data[1][2].shape, train_data[1][3].shape)    
# 第一个维度表示第1个batch
# 第二个维度[0]代表英文个数，[1]代表英文长度
# [2]代表中文词个数，[3]代表中文长度
# 每个batch里的句子长度是不一样的，同一batch里面的句子长度由于填充0使得一样

(64, 11) (64,) (64, 15) (64,)


# Seq2Seq模型

Seq2Seq由编码器和解码器组成，即两个GRU组成。
- 编码器负责句子的编码任务，把句子的含义统一集中在最后一个时间步的隐藏状态中
- 解码器进行翻译

下面的工作是先搭建编码器和解码器的GRU，然后再把两者结合起来组成语言模型。
<img style="float: center;" src="images/15.png" width="70%">

两者都用RNN组成，编码器接收输入的句子，然后进行编码计算，把句子的所有信息综合到最后的一个时间步隐藏状态上，然后作为解码器的初始隐藏状态，开始解码。

## 编码器

编码器接收一个batch（64个句子），每个句子都在它们各自在字典中的位置进行表示，所以输入就是一个[batch_size, seq_len]，之后会经过一个embedding层进行每个单词的词嵌入表示，此时数据维度就变成了[batch_size, seq_len, embed_size]，这个与之前的思路一致，但是有一点不同，就是在embedding之前，需要将句子长度从长到短进行排序，因为这次输入的是序列。
<img style="float: center;" src="images/16.png" width="70%">

虽然形成batch的时候填充0使它们维度一致，但这些无效的0字符在输入GRU时候需要pack掉，因为padding多余的0之后，会导致GRU对它的表示通过了非常多无用的字符，这样得到的句子表示会有误差。

如果想要进行后续的pack操作，需要先对句子进行长到短的排序，这时候经过embedding就得到嵌入之后的数据。之后就是想办法处理这些变长的序列。

在处理之前，先感觉一下变长序列的处理效果，例如yes这句话，如果不加处理，是这样输入到GRU的：
<img style="float: center;" src="images/17.png" width="70%">

而处理之后，是这样的：
<img style="float: center;" src="images/18.png" width="70%">

PyTorch给出了处理这两种情况的两个函数：
- nn.utils.rnn.pack_padded_sequence()
  - 压缩已经经过填充的序列
  - 因为之前把变长序列经过填充变成等长，需要先压缩一下
- nn.utils.rnn.pad_packed_sequence()
  - 填充已经被压缩的序列
  - 需要通过这个函数再变成压缩之前的样子

In [9]:
from torch.autograd import Variable
tensor_in = torch.FloatTensor([[1, 2, 3], [5, 0, 0]]).resize_(2, 3, 1)
tensor_in = Variable(tensor_in)
seq_lenghs = [3, 1]
tensor_in

tensor([[[1.],
         [2.],
         [3.]],

        [[5.],
         [0.],
         [0.]]])

此时如果对其排序（目前已经从长到短排好序了），而真实的数据是从短到长排序，因此需要经历这样的一句：

In [10]:
torch.tensor(seq_lenghs).sort(0, descending=True)    
# 返回两个值，第一个是排好序的数组，第二个是每个元素在原数组里面的位置

torch.return_types.sort(
values=tensor([3, 1]),
indices=tensor([0, 1]))

把填充的数据进行pack，可以发现第二个样本的0都不见了，只留了关键的那些单词。

In [11]:
pack = nn.utils.rnn.pack_padded_sequence(tensor_in, seq_lenghs, batch_first=True)
pack

PackedSequence(data=tensor([[1.],
        [5.],
        [2.],
        [3.]]), batch_sizes=tensor([2, 1, 1]), sorted_indices=None, unsorted_indices=None)

此时如果经过一个RNN（三层的RNN），可以顺便看一下输出h长什么样，因为这个例子基本上完全模拟PyTorch处理变长训练的关键之处，也是解码器的核心部分。

In [12]:
rnn = nn.RNN(1, 2, 3, batch_first=True)   # 输入维度是1(embed_dim)，输出维度是2(2个隐藏单元), 3层
h0 = Variable(torch.randn(3, 2, 2))  # h0的初始状态， (layers_num*direction_nums, batch_size, hidden_size)

out, h = rnn(pack, h0)
out[0].shape, out, h

(torch.Size([4, 2]),
 PackedSequence(data=tensor([[-0.5723, -0.3532],
         [ 0.3066,  0.8889],
         [-0.6507, -0.0175],
         [-0.5896, -0.1284]], grad_fn=<CatBackward0>), batch_sizes=tensor([2, 1, 1]), sorted_indices=None, unsorted_indices=None),
 tensor([[[ 0.3008, -0.2164],
          [ 0.5393, -0.4938]],
 
         [[ 0.8907,  0.8570],
          [-0.4533, -0.5563]],
 
         [[-0.5896, -0.1284],
          [ 0.3066,  0.8889]]], grad_fn=<StackBackward0>))

看h（最后一层的最后一个时间步的隐藏状态），解码器就是以这个作为新的GRU初始隐藏状态的输入，想要获取这个就是h[[-1]]。
<img style="float: center;" src="images/19.png" width="70%">

得到上面out输出之后，接下来就是pad_packed_sequence()

经过这一轮的操作，变成序列合理的处理，先压缩，然后再填充回去就得到和原来一致的维度关系，这也是PyTorch处理变成序列的原理。

In [13]:
unpackded = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
unpackded

(tensor([[[-0.5723, -0.3532],
          [-0.6507, -0.0175],
          [-0.5896, -0.1284]],
 
         [[ 0.3066,  0.8889],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000]]], grad_fn=<TransposeBackward0>),
 tensor([3, 1]))

编码器中，经过这样的操作之后，就可以使得变长序列准确地通过GRU得到每个单词地表示，然后再换回原来的顺序（句子短到长），然后拿到最后地输出和最后一个隐藏状态即可，这就是编码器的前向传播过程。

In [14]:
class PlainEncoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout=0.2):
        super(PlainEncoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        # 第一个维度应该是embed_size，这里为了方便，相等了
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, lengths):  
        # 输入lengths，每个句子是不等长，需要每个句子最后一个时间步的隐藏状态,
        # 因此所以需要知道句子长度，x表示一个batch里面的句子 
        
        # 把batch里面的seq按照长度排序
        # sorted_len表示排好序的数组，sorted_index表示每个元素在原数组位置
        sorted_len, sorted_idx = lengths.sort(0, descending=True)
        x_sorted = x[sorted_idx.long()]  # 句子已经按照seq长度排好序
        embedded = self.dropout(self.embed(x_sorted))  # [batch_size, seq_len, embed_size]
        
        # 处理变长序列
        # data.numpy()是原来张量的克隆，转成numpy数组，相当于clone().numpy()
        # 把变长序列的0都给去掉，之前填充的字符都给压扁
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, sorted_len.long().cpu().data.numpy(), batch_first=True)
        # 得到batch中每个样本的真实隐藏状态
        packed_out, hid = self.rnn(packed_embedded)
        # 填充回去
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        # 让短的句子在前面
        _, original_idx = sorted_idx.sort(0, descending=False)
        # contiguous是为了把不连续的内存单元连续起来 
        out = out[original_idx.long()].contiguous()
        hid = hid[:, original_idx.long()].contiguous()
        
        return out, hid[[-1]]   # 把最后一层的hid给拿出来

## PyTorch中的GRU

torch.nn.GRU(input_size, hidden_size, num_layers, bias, batch_first, dropout, bidirectional)
- input_size：（seq_len, batch, input_size），这个PyTorch默认是seq_len是第一维，如果不设置batch_first维True的时候，就需要把输入转成这样的维度才行。input_size是特征数量。
- h_0：（num_layers\*num_directions, batch, hidden_size），如果不提供的话，默认初始化为0

再看一下这个层的输出：
- output：(seq_len, batch, num_directions\*hidden_size)，output是**最后一层所有时间步的隐藏状态**，seq_len代表所有时间步，batch代表每个样本，num_directions\*hidden_size就是每个方向上隐藏单元个数。
- h_n：(num_layers\*num_directions, batch, hidden_size)，**最后一个时间步所有曾的隐藏状态**，第一个维度是层数和方向数，第二个是样本，第三个是每一层每个时间步的隐藏单元个数。
<img style="float: center;" src="images/20.png" width="70%">

因此，可以很容易拿到最后一层某个时间步的隐藏状态和最后一个时间步某个层的隐藏状态。

## 解码器

也是一个GRU，隐藏状态的初始值来自于编码器最后一个时间步的h，而不是随机初始化的一个h。

编码器这里接收中文输入，需要基于上一个词才能预测出下一个词。

编码器前向传播过程：接受输入是y（中文），大小是[batch_size, seq_len - 1]，这里减一是因为不用全部，基于第一个就能预测第二个，所以基于前n-1个就能预测出全部来。同样也会先embedding，但是embedding之前依然是句子从长到短排序，依然需要压缩还原，与编码器代码几乎一致

In [15]:
# 这个基本上和Encoder是一致的，无非就是初始化的h换成了Encoder之后的h
class PlainDecoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout=0.2):
        super(PlainDecoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, y, y_lengths, hid):
        # y: [batch_size, seq_len-1]
        # 句子从长到短排序
        sorted_len, sorted_idx = y_lengths.sort(0, descending=True)
        y_sorted = y[sorted_idx.long()]
        hid = hid[:, sorted_idx.long()]
        
        # [batch_size, outpout_length, embed_size]
        y_sorted = self.dropout(self.embed(y_sorted))
        
        pack_seq = nn.utils.rnn.pack_padded_sequence(y_sorted, sorted_len.long().cpu().data.numpy(), batch_first=True)
        out, hid = self.rnn(pack_seq, hid)   # 每个有效时间步单词的最后一层的隐藏状态
        unpacked, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)   # [batch, seq_len-1, hidden_size]
        _, original_idx = sorted_idx.sort(0, descending=False)
        output_seq = unpacked[original_idx.long()].contiguous()  # [batch, seq_len-1, hidden_size]
        
        hid = hid[:, original_idx.long()].contiguous()   # [1, batch, hidden_size]
        output = F.log_softmax(self.out(output_seq), -1)        
        # [batch, seq_len-1, vocab_size]   每个样本每个时间步长都有一个vocab_size的维度长度，表示每个单词的概率
        
        return output, hid

## Seq2Seq模型

把编码器和解码器连起来组成一个简单的Seq2Seq模型。

前向传播就是编码器对英文句子编码计算，得到最后一个时间步的h，这个可以理解蕴涵着输入句子的信息，然后到解码器中解码，得到输出output

In [16]:
class PlainSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(PlainSeq2Seq, self).__init__()
        self.encoder = encoder   
        self.decoder = decoder
        
    def forward(self, x, x_lengths, y, y_lengths):
        encoder_out, hid = self.encoder(x, x_lengths)  # encoder 进行编码
        output, hid = self.decoder(y, y_lengths, hid)  # deocder 负责解码
        return output, None

    # 对句子进行翻译，max_length句子的最大长度
    # 测试最后模型效果的时候用，与Seq2Seq本身没有啥关系
    def translate(self, x, x_lengths, y, max_length=10):
        encoder_out, hid = self.encoder(x, x_lengths)   # 解码
        preds = []
        batch_size = x.shape[0]
        attns = []
        for i in range(max_length):   
            output, hid = self.decoder(y, torch.ones(batch_size).long().to(y.device), hid=hid)
            y = output.max(2)[1].view(batch_size, 1)
            preds.append(y)
        
        return torch.cat(preds, 1), None

## 模型训练

需要写一个损失函数，因为句子是变长的，需要用mask处理那些无用的填充字符，类似于写一个NLLoss的东西。

In [17]:
# masked cross entropy loss
class LanguageModelCriterion(nn.Module):
    def __init__(self):
        super(LanguageModelCriterion, self).__init__()
    
    def forward(self, input, target, mask):
        # input: [batch_size, seq_len, vocab_size]    每个单词的可能性
        input = input.contiguous().view(-1, input.size(2))   # [batch_size*seq_len-1, vocab_size]
        target = target.contiguous().view(-1, 1)    #  [batch_size*seq_len-1, 1]
        
        mask = mask.contiguous().view(-1, 1)   # [batch_size*seq_len-1, 1]
        # 在每个vocab_size维度取正确单词的索引，里面有很多是填充的，mask去掉这些填充
        output = -input.gather(1, target) * mask
        # 这个其实在写一个NLloss，也就是sortmax的取负号
        output = torch.sum(output) / torch.sum(mask)
        
        return output  # [batch_size*seq_len-1, 1]

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dropout = 0.2
hidden_size = 100
encoder = PlainEncoder(vocab_size=en_total_words, hidden_size=hidden_size, dropout=dropout)
decoder = PlainDecoder(vocab_size=cn_total_words, hidden_size=hidden_size, dropout=dropout)

model = PlainSeq2Seq(encoder, decoder)
model = model.to(device)
loss_fn = LanguageModelCriterion().to(device)
optimizer = torch.optim.Adam(model.parameters())

In [19]:
# 定义训练和验证函数
def evaluate(model, data):
    model.eval()
    total_num_words = total_loss = 0.
    with torch.no_grad():
        for it, (mb_x, mb_x_len, mb_y, mb_y_len) in enumerate(data):
            mb_x = torch.from_numpy(mb_x).to(device).long()    # 这个是一个batch的英文句子 大小是[batch_size, seq_len]
            mb_x_len = torch.from_numpy(mb_x_len).to(device).long()    # 每个句子的长度
            mb_input = torch.from_numpy(mb_y[:, :-1]).to(device).long()  # 解码器那边的输入， 输入一个单词去预测另外一个单词
            mb_output = torch.from_numpy(mb_y[:, 1:]).to(device).long()   # 解码器那边的输出  [batch_size, seq_len-1]
            mb_y_len = torch.from_numpy(mb_y_len-1).to(device).long()  # 这个减去1， 因为没有了最后一个  [batch_size, seq_len-1]
            mb_y_len[mb_y_len<=0] =  1   # 这句话是为了以防出错
            
            mb_pred, attn = model(mb_x, mb_x_len, mb_input, mb_y_len)
            
            mb_out_mask = torch.arange(mb_y_len.max().item(), device=device)[None, :] < mb_y_len[:, None]  
            # [batch_size, mb_y_len.max()], 上面是bool类型， 下面是float类型， 只计算每个句子的有效部分， 填充的那部分去掉
            mb_out_mask = mb_out_mask.float()  # [batch_size, seq_len-1]  因为mb_y_len.max()就是seq_len-1
            
            loss = loss_fn(mb_pred, mb_output, mb_out_mask)
            
            num_words = torch.sum(mb_y_len).item()
            total_loss += loss.item() * num_words
            total_num_words += num_words
    print('Evaluation loss', total_loss / total_num_words)


def train(model, data, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        total_num_words = total_loss = 0.
        for it, (mb_x, mb_x_len, mb_y, mb_y_len) in enumerate(data):
            mb_x = torch.from_numpy(mb_x).to(device).long()
            mb_x_len = torch.from_numpy(mb_x_len).to(device).long()
            mb_input = torch.from_numpy(mb_y[:, :-1]).to(device).long()
            mb_output = torch.from_numpy(mb_y[:, 1:]).to(device).long()
            mb_y_len = torch.from_numpy(mb_y_len-1).to(device).long()
            mb_y_len[mb_y_len<=0] = 1
            
            mb_pred, attn = model(mb_x, mb_x_len, mb_input, mb_y_len)
            
            mb_out_mask = torch.arange(mb_y_len.max().item(), device=device)[None, :] < mb_y_len[:, None]
            mb_out_mask = mb_out_mask.float()
            
            loss = loss_fn(mb_pred, mb_output, mb_out_mask)
            
            num_words = torch.sum(mb_y_len).item()
            total_loss += loss.item() * num_words
            total_num_words += num_words
            
            # 更新
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)     # 这里防止梯度爆炸， 这是和以往不太一样的地方
            optimizer.step()
            
            if it % 100 == 0:
                print('Epoch', epoch, 'iteration', it, 'loss', loss.item())
            
        print('Epoch', epoch, 'Training loss', total_loss / total_num_words)
        if epoch % 5 == 0:
            evaluate(model, dev_data)
        
# 训练
train(model, train_data, num_epochs=20)

Epoch 0 iteration 0 loss 9.35302448272705
Epoch 0 iteration 100 loss 6.020293235778809
Epoch 0 iteration 200 loss 4.854287147521973
Epoch 0 Training loss 5.889250806130928
Evaluation loss 5.210167619484115
Epoch 1 iteration 0 loss 4.4433417320251465
Epoch 1 iteration 100 loss 5.562403202056885
Epoch 1 iteration 200 loss 4.349039554595947
Epoch 1 Training loss 4.958273569633383
Epoch 2 iteration 0 loss 3.9863054752349854
Epoch 2 iteration 100 loss 5.250918388366699
Epoch 2 iteration 200 loss 4.048994064331055
Epoch 2 Training loss 4.62348482656324
Epoch 3 iteration 0 loss 3.6867971420288086
Epoch 3 iteration 100 loss 5.00648832321167
Epoch 3 iteration 200 loss 3.8102409839630127
Epoch 3 Training loss 4.375772453130721
Epoch 4 iteration 0 loss 3.45863676071167
Epoch 4 iteration 100 loss 4.806687831878662
Epoch 4 iteration 200 loss 3.646423578262329
Epoch 4 Training loss 4.166808362241544
Epoch 5 iteration 0 loss 3.2544803619384766
Epoch 5 iteration 100 loss 4.627119064331055
Epoch 5 iter

mask制作方式：遮盖掉那些填充的那部分字符，这时候计算损失时，就只计算有效部分。

假设有5个句子，长度分别是：3，8，10，2，1，可以先根据最大长度10生成0-10的一个数组

In [20]:
m = torch.tensor([3, 8, 10, 2, 1])
torch.arange(m.max().item())[None, :]   # tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

这个就是上面的：
- mb_out_mask = torch.arange(mb_y_len.max().item(), device=device)[None, :] < mb_y_len[:, None]
- [None, :]操作扩展第一维度，类似[np.newaxis, :]
- 后半部分：m[:, None]

In [21]:
m[:, None]

tensor([[ 3],
        [ 8],
        [10],
        [ 2],
        [ 1]])

此时，如果两边执行小于操作，就会得到如下结果：

In [22]:
torch.arange(m.max().item())[None, :] < m[:, None]

tensor([[ True,  True,  True, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True, False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False, False, False]])

此时，每一行代表一个句子，而True部分代表句子真实长度，False代表填充部分，这时候再去取预测单词的具体位置，就可以把无用的字符过滤掉

## 结果预测

In [25]:
def translate_dev(i):
    en_sent = " ".join([inv_en_dict[w] for w in dev_en[i]])  #原来的英文
    print(en_sent)
    cn_sent = " ".join([inv_cn_dict[w] for w in dev_cn[i]])  #原来的中文
    print("".join(cn_sent))
 
    # 一条句子
    mb_x = torch.from_numpy(np.array(dev_en[i]).reshape(1, -1)).long().to(device)
    mb_x_len = torch.from_numpy(np.array([len(dev_en[i])])).long().to(device)
    bos = torch.Tensor([[cn_dict["BOS"]]]).long().to(device)  # shape:[1,1], [[2]]
    
    # y_lengths: [[2]], 一个句子
    translation, attn = model.translate(mb_x, mb_x_len, bos)  # [1, 10]
    # 映射成中文
    translation = [inv_cn_dict[i] for i in translation.data.cpu().numpy().reshape(-1)]
    trans = []
    for word in translation:
        if word != "EOS":
            trans.append(word)
        else:
            break
    print("".join(trans))           #翻译后的中文

In [26]:
for i in range(100, 120):
    translate_dev(i)
    print()

BOS you have nice skin . EOS
BOS 你 的 皮膚 真好 。 EOS
你看上去。

BOS you 're UNK correct . EOS
BOS 你 UNK 正确 。 EOS
你是你的。

BOS everyone admired his courage . EOS
BOS 每個 人 都 佩服 他 的 勇氣 。 EOS
每個人都認識他。

BOS what time is it ? EOS
BOS 几点 了 ？ EOS
它是什么？

BOS i 'm free tonight . EOS
BOS 我 今晚 有空 。 EOS
我很忙。

BOS here is your book . EOS
BOS 這是 你 的 書 。 EOS
你是个美人。

BOS they are at lunch . EOS
BOS 他们 在 吃 午饭 。 EOS
他们是个好人。

BOS this chair is UNK . EOS
BOS 這把 椅子 UNK 。 EOS
这是一个生和死的。

BOS it 's pretty heavy . EOS
BOS 它 UNK 。 EOS
它是什么。

BOS many attended his funeral . EOS
BOS 很多 人 都 参加 了 他 的 UNK 。 EOS
他的意見有道理。

BOS training will be provided . EOS
BOS 会 有 训练 。 EOS
保持安静。

BOS someone is watching you . EOS
BOS 有人 在 看 著 你 。 EOS
汤姆在做。

BOS i slapped his face . EOS
BOS 我 摑 了 他 的 臉 。 EOS
我的房子。

BOS i like UNK music . EOS
BOS 我 喜歡 流行 音樂 。 EOS
我喜歡爵士樂。

BOS tom had no children . EOS
BOS Tom 沒有 孩子 。 EOS
汤姆在波士顿。

BOS please lock the door . EOS
BOS 請 把 UNK 上 。 EOS
請說話。

BOS tom has calmed down . EOS
BOS 汤姆 冷静下来 了 。 EOS
汤姆挺受欢迎。

B

# Attention模型

Luong的Attention模型：
- 论文：《Effective Approaches to Attention-based Neural Machine Translation》
- https://arxiv.org/pdf/1508.04025.pdf

<img style="float: center;" src="images/21.png" width="70%">

Attention与之前模型的不同之处在于这里有一个$c_t$，即上下文向量。

之前的简单结构，是编码器编码之后拿到最后一个隐藏状态直接给解码器，解码器拿到h，拿到前一步的$y_{t-1}$，就可以进行$y_t$的翻译，但是这里不一样。

解码器的最后输出部分，是经过一个Attention机制，预测当前输出的时候都会给所有输入加一个权重来表示输入与当前输出的一个关联关系，权重越大，说明当前输入对当前输出的重要性越大。

这个权重，根据论文这样得到：
<img style="float: center;" src="images/22.png" width="70%">

这里的score，有三种方式，这里选用第一种直接点积，表示二者相似程度。
<img style="float: center;" src="images/23.png" width="70%">

这里的重点是Attention机制，儿Attention机制就是在求一个权重，权重乘以输入就得到一个上下文向量，这个上下文向量又关注了蕴涵的输入信息。上下文向量和当前输出的隐藏状态合起来再经过tanh，再经过一个线性层就得到了最终的输出结果。

## 编码器

编码器部分与之前一致，任何是把输入文字传入embedding层和GRU层，转换成一些hidden states作为后续的上下文向量。

In [27]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, enc_hidden_size, dec_hidden_size, dropout=0.2):
        super(Encoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, enc_hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(enc_hidden_size*2, dec_hidden_size)
        
    def forward(self, x, lengths):
        sorted_len, sorted_idx = lengths.sort(0, descending=True)
        x_sorted = x[sorted_idx.long()]
        embedded = self.dropout(self.embed(x_sorted))   # [batch_size, seq_len, embed_size]
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, sorted_len.long().cpu().data.numpy(), batch_first=True)
        packed_out, hid = self.rnn(packed_embedded)
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)  # [batch_size, seq_len, 2*enc_hidden_size]
        _, original_idx = sorted_idx.sort(0, descending=False)
        out = out[original_idx.long()].contiguous()   # [batch_size, seq_len, 2*enc_hidden_size]
        hid = hid[:, original_idx.long()].contiguous()   # [2, batch_size, enc_hidden_size]
        
        hid = torch.cat([hid[-2], hid[-1]], dim=1)   # 双向的GRU，这里是最后一个状态， 联结起来  [batch_size, 2*enc_hidden_size]
        hid = torch.tanh(self.fc(hid)).unsqueeze(0)  # [1, batch_size, dec_hidden_size]
        
        return out, hid

## Attention机制

In [28]:
class Attention(nn.Module):
    def __init__(self, enc_hidden_size, dec_hidden_size):
        super(Attention, self).__init__()
        
        self.enc_hidden_size = enc_hidden_size
        self.dec_hidden_size = dec_hidden_size
        
        self.linear_in = nn.Linear(enc_hidden_size*2, dec_hidden_size, bias=False)
        self.linear_out = nn.Linear(enc_hidden_size*2+dec_hidden_size, dec_hidden_size)
    
    def forward(self, output, encoder_output, mask):
        # output: [batch_size, seq_len_y-1, dec_hidden_size]  这个output 是decoder的每个时间步输出的隐藏状态
        # encoder_output: [batch_size, seq_len_x, 2*enc_hidden_size]
        batch_size = output.size(0)
        output_len = output.size(1)
        input_len = encoder_output.size(1)
        
        context_in = self.linear_in(encoder_output.view(batch_size*input_len, -1))  # [batch_size*seq_len_x,dec_hidden_size]
        context_in = context_in.view(batch_size, input_len, -1)  # [batch_size, seq_len_x, dec_hidden_size]
        context_in = context_in.transpose(1, 2)   # [batch_size, dec_hidden_size, seq_len_x]
        
        attn = torch.bmm(output, context_in)  # [batch_size, seq_len_y-1, seq_len_x]
        # 这个东西就是求得当前时间步的输出output和所有输入相似性关系的一个得分score , 下面就是通过softmax把这个得分转成权重
        attn = F.softmax(attn, dim=2)    # 此时第二维度的数字全都变成了0-1之间的数， 越大表示当前的输出output与哪个相关程度越大
        
        context = torch.bmm(attn, encoder_output)   # [batch_size, seq_len_y-1, 2*enc_hidden_size]
        
        output = torch.cat((context, output), dim=2)  # [batch_size, seq_len_y-1, 2*enc_hidden_size+dec_hidden_size]
        
        output = output.view(batch_size*output_len, -1)   # [batch_size*seq_len_y-1, 2*enc_hidden_size+dec_hidden_size]
        output = torch.tanh(self.linear_out(output))     # [batch_size*seq_len_y-1, dec_hidden_size]
        output = output.view(batch_size, output_len, -1)  # [batch_size, seq_len_y-1, dec_hidden_size]
        
        return output, attn

## 解码器

decoder会根据已经翻译好的句子内容和当前的context vector，来决定下一个输出的单词，这里与之前不同的之处在于最后输出经过了一个Attention

In [29]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, enc_hidden_size, dec_hidden_size, dropout=0.2):
        super(Decoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.attention = Attention(enc_hidden_size, dec_hidden_size)
        self.rnn = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.out = nn.Linear(dec_hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def create_mask(self, x_len, y_len):
        # a mask of shape x_len*y_len
        x_mask = torch.arange(x_len.max(), device=x_len.device)[None, :] < x_len[:, None]
        y_mask = torch.arange(y_len.max(), device=x_len.device)[None, :] < y_len[:, None]
        
        x_mask = x_mask.float()
        y_mask = y_mask.float()
        mask = (1 - x_mask[:, :, None] * y_mask[:, None, :]).byte()
        return mask
    
    def forward(self, encoder_out, encoder_out_lengths, y, y_lengths, hid):
        sorted_len, sorted_idx = y_lengths.sort(0, descending=True)
        y_sorted = y[sorted_idx.long()]   # 句子从长到短排序
        hid = hid[:, sorted_idx.long()]
        
        y_sorted = self.dropout(self.embed(y_sorted))     # [batch_size, output_length, embed_size]
        
        packed_seq = nn.utils.rnn.pack_padded_sequence(y_sorted, sorted_len.long().cpu().data.numpy(), batch_first=True)
        out, hid = self.rnn(packed_seq, hid)
        unpacked, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        _, original_idx = sorted_idx.sort(0, descending=False)
        output_seq = unpacked[original_idx.long()].contiguous()   # [batch_size, seq_len_y-1, dec_hidden_size]
        hid = hid[:, original_idx.long()].contiguous()
        
        mask = self.create_mask(y_lengths, encoder_out_lengths)
        
        output, attn = self.attention(output_seq, encoder_out, mask)
        output = F.log_softmax(self.out(output), -1)
        
        return output, hid, attn

## Seq2Seq+Attention

In [30]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, x, x_lengths, y, y_lengths):
        encoder_out, hid = self.encoder(x, x_lengths)
        output, hid, attn = self.decoder(encoder_out, x_lengths, y, y_lengths, hid)
        
        return output, attn
    
    def translate(self, x, x_lengths, y, max_length=100):
        encoder_out, hid = self.encoder(x, x_lengths)
        preds = []
        batch_size = x.shape[0]
        attns = []
        for i in range(max_length):
            output, hid, attn = self.decoder(encoder_out, x_lengths, y, torch.ones(batch_size).long().to(y.device), hid)
            y = output.max(2)[1].view(batch_size, 1)
            preds.append(y)
            attns.append(attn)
        
        return torch.cat(preds, 1), torch.cat(attns, 1)

## 模型训练

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dropout = 0.2
hidden_size = 100
embed_size = 20
encoder = Encoder(vocab_size=en_total_words, embed_size=embed_size, enc_hidden_size=hidden_size, dec_hidden_size=hidden_size, dropout=dropout)
decoder = Decoder(vocab_size=cn_total_words, embed_size=embed_size, enc_hidden_size=hidden_size, dec_hidden_size=hidden_size, dropout=dropout)

model = Seq2Seq(encoder, decoder)
model = model.to(device)
loss_fn = LanguageModelCriterion().to(device)
optimizer = torch.optim.Adam(model.parameters())

In [34]:
train(model, train_data, num_epochs=20)

Epoch 0 iteration 0 loss 9.34937572479248
Epoch 0 iteration 100 loss 6.319538116455078
Epoch 0 iteration 200 loss 5.25015115737915
Epoch 0 Training loss 6.127085370002386
Evaluation loss 5.603679679314572
Epoch 1 iteration 0 loss 4.906866550445557
Epoch 1 iteration 100 loss 5.99578857421875
Epoch 1 iteration 200 loss 4.916123390197754
Epoch 1 Training loss 5.429204816858988
Epoch 2 iteration 0 loss 4.606022834777832
Epoch 2 iteration 100 loss 5.843180179595947
Epoch 2 iteration 200 loss 4.719799041748047
Epoch 2 Training loss 5.2283853432862495
Epoch 3 iteration 0 loss 4.401919841766357
Epoch 3 iteration 100 loss 5.7497100830078125
Epoch 3 iteration 200 loss 4.547033786773682
Epoch 3 Training loss 5.082325355668237
Epoch 4 iteration 0 loss 4.234122276306152
Epoch 4 iteration 100 loss 5.561656951904297
Epoch 4 iteration 200 loss 4.357476234436035
Epoch 4 Training loss 4.909747890392661
Epoch 5 iteration 0 loss 4.047961235046387
Epoch 5 iteration 100 loss 5.4158711433410645
Epoch 5 itera

In [35]:
for i in range(100, 120):
    translate_dev(i)
    print()

BOS you have nice skin . EOS
BOS 你 的 皮膚 真好 。 EOS
你有狗。

BOS you 're UNK correct . EOS
BOS 你 UNK 正确 。 EOS
你是个很好。

BOS everyone admired his courage . EOS
BOS 每個 人 都 佩服 他 的 勇氣 。 EOS
他的父母都知道了。

BOS what time is it ? EOS
BOS 几点 了 ？ EOS
那是什么？

BOS i 'm free tonight . EOS
BOS 我 今晚 有空 。 EOS
我很喜欢。

BOS here is your book . EOS
BOS 這是 你 的 書 。 EOS
你的狗是你的。

BOS they are at lunch . EOS
BOS 他们 在 吃 午饭 。 EOS
他們有這裡。

BOS this chair is UNK . EOS
BOS 這把 椅子 UNK 。 EOS
这是一個好的。

BOS it 's pretty heavy . EOS
BOS 它 UNK 。 EOS
它是危險的。

BOS many attended his funeral . EOS
BOS 很多 人 都 参加 了 他 的 UNK 。 EOS
他在狗上了。

BOS training will be provided . EOS
BOS 会 有 训练 。 EOS
讓我們的时候。

BOS someone is watching you . EOS
BOS 有人 在 看 著 你 。 EOS
你的狗是你的。

BOS i slapped his face . EOS
BOS 我 摑 了 他 的 臉 。 EOS
我的祖父他。

BOS i like UNK music . EOS
BOS 我 喜歡 流行 音樂 。 EOS
我不喜欢。

BOS tom had no children . EOS
BOS Tom 沒有 孩子 。 EOS
汤姆在笑。

BOS please lock the door . EOS
BOS 請 把 UNK 上 。 EOS
请清洗。

BOS tom has calmed down . EOS
BOS 汤姆 冷静下来 了 。 EOS
汤姆在波士顿。

B